# Promote DTA to Library Major Version

This notebook promotes a DTA major version to a new library major version.

**Use Case:**
When a DTA's approved version should become the new production standard for the library.

**Parameters:**
- `dta_id` - DTA identifier
- `dta_version` - DTA major version to promote (e.g., "1.0-DTA_A-v1.0")
- `library_type` - Library type (e.g., transfer_variables)

**Result:**
Creates a new library major version (e.g., "2.0") and closes the previous library major using SCD Type 2.


In [ ]:
# Cell 1: Imports and Setup
import json
from clinical_data_standards_framework.versioning import (
    promote_to_library_major,
    get_current_library_version,
    get_all_versions
)

print("="*80)
print("PROMOTE DTA TO LIBRARY MAJOR VERSION")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Config: {len(library_tables)} library table(s)")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters (job-level parameters only)

dbutils.widgets.text("dta_id", "", "DTA ID")
dbutils.widgets.text("dta_version", "", "DTA Version to Promote")
dbutils.widgets.text("library_type", "", "Library Type (defaults from config)")

dta_id = dbutils.widgets.get("dta_id")
dta_version = dbutils.widgets.get("dta_version")
library_type_param = dbutils.widgets.get("library_type")

# Default to first library type from config if not specified
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

print(f"\nDTA ID: {dta_id}")
print(f"DTA Version: {dta_version}")
print(f"Library Type: {library_type} (from {'param' if library_type_param else 'config'})")

# Validate required parameters
if not dta_id or not dta_version:
    raise ValueError("dta_id and dta_version parameters are required")


In [ ]:
# Cell 4: Configure Table Names (from config)

# Build library table map from config
library_table_map = {}
for lib_config in library_tables:
    lib_type = lib_config.get('library_type')
    lib_name = lib_config.get('name')
    lib_schema = lib_config.get('schema', gold_schema)
    if lib_type and lib_name:
        library_table_map[lib_type] = f"{catalog}.{lib_schema}.{lib_name}"

if library_type not in library_table_map:
    available = ", ".join(library_table_map.keys())
    raise ValueError(f"Unknown library_type: {library_type}. Available: {available}")

library_table = library_table_map[library_type]
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"Library Table: {library_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 5: Show Current Library Version

print("\n" + "="*80)
print("Current Library Major Version")
print("="*80)

current_lib = get_current_library_version(spark, library_table)
current_count = current_lib.count()
print(f"\nCurrent library has {current_count} records")

# Show sample
current_lib.select("definition_hash", "transfer_variable_name", "library_version", "is_current").show(5, truncate=False)


In [ ]:
# Cell 6: Execute Promotion

print("\n" + "="*80)
print(f"Promoting: {dta_version} -> Library Major")
print("="*80)

new_version = promote_to_library_major(
    spark=spark,
    library_table=library_table,
    registry_table=registry_table,
    dta_id=dta_id,
    dta_version=dta_version,
    library_type=library_type,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id
)

print(f"\nNew Library Major Version: {new_version}")


In [ ]:
# Cell 7: Set Task Values and Show Summary

dbutils.jobs.taskValues.set(key="new_library_version", value=new_version)
dbutils.jobs.taskValues.set(key="promoted_from", value=dta_version)

print("\n" + "="*80)
print("Library Major Version History")
print("="*80)

all_versions = get_all_versions(spark, registry_table, library_type)
all_versions.filter("version_type = 'LIBRARY_MAJOR'").select(
    "version_tag", "status", "record_count", "parent_version", "created_ts"
).show(truncate=False)
